In [ ]:
import star_functions as nana

In [ ]:
import importlib
importlib.reload(nana)

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.io import ascii
import numpy as np
from astropy import units as u

In [ ]:
star_files = glob.glob("*.dat")

In [ ]:
print(star_files)
#kplr010536147

In [ ]:
def do_one_star(star_fn, exptime = 'long'):
    
    star = star_fn[:13] #brittle

    print("working on ", star)
    
    #pipeline 
    lc, delta_f, sampling_time, exptime = nana.star(star, exptime = exptime)
    print("delta_f:", delta_f)
    print("sampling_time:", sampling_time)
    if lc is None:
        return 
    f_max = (3 / (2*sampling_time))
    f_min = delta_f/3
    t_fit, flux_fit, weight_fit = nana.mask_vals(lc)
    freq_full, power_full = nana.pg_full(f_min, f_max, lc)
    freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
    indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
    refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
    fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
    regions, final_freqs, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit, flux_fit, weight_fit, exptime)
    '''
    plt.plot(freq_full, power_full, 'k.')
    plt.scatter(final_freqs, refined_power, color = 'red', marker = 'o')
    plt.show()
    
    plt.plot(freq_mini, power_mini, 'g-')
    plt.semilogy()
    plt.show()
    '''
    a_early, a_late, b_early, b_late = nana.check_coherence(t_fit, flux_fit, weight_fit, final_freqs, exptime)
    rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit)
    delta_chi2s = nana.null_chi_squared(flux_fit, weight_fit) - chi2s
    sharpnesses = nana.sharpness(second_derivatives, refined_power)
    
    '''
    data = Table()


    data['modes'] = final_freqs
    data['regions'] = regions
    data['corresponding_freq_in_region_A'] = refined_freq
    data['sharpness'] = sharpnesses
    data['delta_chi_squares'] = delta_chi2s
    data['rate_of_change_of_phase'] = rate_of_phase
    data['rate_of_change_of_amplitude'] = rate_of_amp
    
    ascii.write(
        data,
        star + '.csv',
        overwrite=True,
        format="csv",
        formats={
            "modes": "{:.10f}",
            "corresponding_freq_in_region_A": "{:.5f}",
            "sharpness": "{:.7e}",
            "delta_chi_squares": "{:.7e}",
            "rate_of_change_of_phase": "{:.7e}",
            "rate_of_change_of_amplitude": "{:.7e}"
        }
    )
    '''

    return
    

In [ ]:
##out of spec stars
#kplr006462033
#kplr010536147
#kplr008459899
#kplr003459297
#kplr003240411
#kplr012258330
#

In [ ]:
#out of spec stars
#kplr006462033
#kplr010536147
#kplr008459899
#kplr003240411
#kplr012258330
#kplr008766405

In [ ]:
#for star in star_files:
  #  do_one_star(star, exptime = exptime)

In [ ]:
def check_coherence2(ts, ys, weights, oms, T):

    Qs = [[[ ] , [ ]] for i in range(4)]

    ts_median = np.median(ts)
    t_first = np.median(ts[ts < ts_median])
    t_third = np.median(ts[ts >= ts_median])

    mask_q1 = (ts < ts_median) & (ts < t_first)
    mask_q2 = (ts < ts_median) & (ts >= t_first)
    mask_q3 = (ts >= ts_median) & (ts < t_third)
    mask_q4 = (ts >= ts_median) & (ts >= t_third)

    cases = [mask_q1, mask_q2, mask_q3, mask_q4]
    
    for om in oms:
        for q, I in zip(Qs, cases):
            A = nana.integral_design_matrix(ts[I], om, T)
            pars, _ =nana.weighted_least_squares_new(A, ys[I], weights[I])
            q[0].append(pars[1])
            q[1].append(pars[2])

    return Qs[0], Qs[1], Qs[2], Qs[3]
            



In [ ]:
#og KIC 520 star star
lc, delta_f, sampling_time, exptime520 = nana.star('KIC 003336252')
print("delta_f:", delta_f)
print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit520, flux_fit520, weight_fit520 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs520, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit520, flux_fit520, weight_fit520, exptime520)

plt.plot(freq_full, power_full, 'k.')
plt.scatter(final_freqs520, refined_power, color = 'red', marker = 'o')
plt.show()

plt.plot(freq_mini, power_mini, 'g-')
plt.semilogy()
plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit520)
delta_chi2s = nana.null_chi_squared(flux_fit520, weight_fit520) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
#og KIC 520 star star
lc, delta_f, sampling_time, exptime520 = nana.star('KIC 5202905')
print("delta_f:", delta_f)
print("sampling_time:", sampling_time)
#if lc is None:
    #return 
f_max = (3 / (2*sampling_time))
f_min = delta_f/3
t_fit520, flux_fit520, weight_fit520 = nana.mask_vals(lc)
freq_full, power_full = nana.pg_full(f_min, f_max, lc)
freq_mini, power_mini = nana.pg_mini(f_min, f_max, lc )
indices = nana.get_filtered_peaks(12, freq_mini, power_mini)
refined_freq, refined_power, second_derivatives = nana.refine_peaks(freq_mini, power_mini, indices)
fc = nana.folding_freq(delta_f, freq_full, power_full, sampling_time, False)
regions, final_freqs520, chi2s = nana.region_and_freq(indices, fc, f_min, freq_mini, power_mini, t_fit520, flux_fit520, weight_fit520, exptime520)

plt.plot(freq_full, power_full, 'k.')
plt.scatter(final_freqs520, refined_power, color = 'red', marker = 'o')
plt.show()

plt.plot(freq_mini, power_mini, 'g-')
plt.semilogy()
plt.show()

a_early, a_late, b_early, b_late = nana.check_coherence(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
rate_of_phase, rate_of_amp = nana.change_in_phase_and_amp(a_early, a_late, b_early, b_late, t_fit520)
delta_chi2s = nana.null_chi_squared(flux_fit520, weight_fit520) - chi2s
sharpnesses = nana.sharpness(second_derivatives, refined_power)

In [ ]:
arrays = check_coherence_test(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)
#print(arrays)

In [ ]:
import matplotlib.ticker as ticker

In [ ]:
def splitting(ts, K, jackknife = True):
    '''
    # splitting()
    Produce either disjoint or jackknife subsamples.
    
    ## Inputs:
    - `ts`: shape `(N, )` array of times (not necesarily ordered)
    - `K`: number of subsamples to return
    - `jackknife`: `True` for jackknife subsamples; otherwise disjoint
    
    ## Output: 
    - `masks`: shape `(K, N)` boolean array; `True` if in subsample

    ## Comment:
    - Relies on numpy conventions about relationships between integers and bools.
    '''
    N = len(ts)
    indices = np.argsort(ts)
    split_indices = np.array_split(indices, K)
    if jackknife:
        masks = np.ones((K, N), dtype=bool) # make everything True by default
        for i, idx in enumerate(split_indices):
            masks[i, idx] = False # remove subsample
    else:
        masks = np.zeros((K, N), dtype=bool) # make everything False by default
        for i, idx in enumerate(split_indices):
            masks[i, idx] = True # add subsample
    return masks

In [ ]:
all, half, quartiles, eighths = coherence_all(t_fit520, flux_fit520, weight_fit520, final_freqs520, exptime520)

In [ ]:
fig, axes = plt.subplots(3, 4, figsize=(16, 9))

for idx, (ax, points1, points2, points3, points4) in enumerate(zip(axes.flat, all, half, quartiles, eighths)):
    a_all = points1[0]  h
    b_all = points1[1]  

    a_half = points2[:, 0]  
    b_half = points2[:, 1]  

    a_quarter = points3[:, 0]  
    b_quarter = points3[:, 1] 

    a_eighth = points4[:, 0]
    b_eighth= points4[:, 1]

    ax.scatter(a_half, b_half, color='green', marker='*', label='Halves')
    ax.scatter(a_quarter, b_quarter, color='blue', marker='+', label='Quarters')
    ax.scatter(a_eighth, b_eighth, color='k', marker='.', label='Eighths (Jackknife)')
    ax.scatter(a_all, b_all, color='red', marker='x', label='All Data')

    if idx == 0:
        ax.legend()
    
    ax.axvline(0, color = 'k')
    ax.axhline(0, color = 'k')
    ax.set_title(final_freqs520[idx])
    ax.set_xlabel("a points")
    ax.set_ylabel("b points")
    ax.grid(True)
    ax.ticklabel_format(style='sci', scilimits=(-3, 3), axis='both')
    ax.axis('equal')



plt.tight_layout()


In [ ]:
###FINAL COHERENCE TEST!!!!
def coherence_all(ts, ys, weights, final_freq, T):
    oms = np.array([f * 2 * np.pi for f in final_freq])
    all = np.zeros((len(oms), 2))

    for idx, om in enumerate(oms):
        A = nana.integral_design_matrix(ts, om, T)
        pars, _ = nana.weighted_least_squares_new(A, ys, weights)
        all[idx][0] = pars[1]
        all[idx][1] = pars[2]
        

    splits = np.array([2, 4, 8])
    results = [np.zeros((len(oms), n, 2)) for n in splits]
    
    for split, result in zip(splits, results):     

        jack = (split == 8)
        masks = splitting(ts, split, jackknives = jack)
        for idx, om in enumerate(oms):
            for i, mask in enumerate(masks):
                A = nana.integral_design_matrix(ts[mask], om, T)
                pars, _ = nana.weighted_least_squares_new(A, ys[mask], weights[mask])
                result[idx][i][0] = pars[1] #a
                result[idx][i][1] = pars[2] #b

    return all, results[0], results[1], results[2] #all, half, quarter, eighth (+jacknives)
    